In [1]:
# coding: utf-8
"""
ラベル付きレビューを形態素解析して学習コーパスを作成
1文ずつ形態素解析したものと、口コミごとに形態素解析したものの2種類を作成
有向グラフ用のエッジリストも作成
"""
import MeCab
import csv
import collections
import pickle

def parsing(sentence):
    mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞":
            if features[6] == "*":
                list_words.append(res.surface)
            else:
                list_words.append(features[6])
        res = res.next
    return list_words

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def readtsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f, delimiter='\t')
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def writedump(arr, path):
    f = open(path, "w")
    pickle.dump(arr, f)
    f.close()

### コーパス作成
1. 同じユーザーの口コミを1文にまとめる
2. ユーザーごとに形態素解析したリストを作成
3. 高頻度、低頻度語を削除するために、除去語リストを作成する
4. 除去

In [62]:
list_sentences = readcsv("./files/list_sentence_user_label.csv")
# 同じユーザーの口コミを1文にまとめる
counter = 0
list_sentences_tmp = []
list_sentences_rev = []
for row in list_sentences:
    if int(row[1]) == counter:
        list_sentences_tmp.append(row[0])
    else:
        counter += 1
        list_sentences_rev.append(list_sentences_tmp)
        list_sentences_tmp = []

list_sentences_rev = ["。".join(row) for row in list_sentences_rev]

# 形態素解析したリストを作成
list_words = [parsing(row) for row in list_sentences_rev]

# 各単語の頻度を数え上げる
list_words_collection = []
for row in list_words:
    list_words_collection.extend(row)
list_words_collection = collections.Counter(list_words_collection).items()

# 除去語リストを作成
list_words_remove = []
for row in list_words_collection:
    if row[1] < 3 or row[1] > 100:
        list_words_remove.append(row[0])

# 除去語リスト内の単語を削除
list_words_rev = []
for row in list_words:
    list_words_tmp = []
    for word in row:
        if word in list_words_remove:
            pass
        else:
            list_words_tmp.append(word)
    list_words_rev.append(list_words_tmp)

### コーパスから有向エッジリストを作成し、保存

In [63]:
# エッジリストを作成
list_edgelist = []
for row in list_words_rev:
    for j in range(len(row) - 1):
        list_edgelist.append([row[j], row[j+1]])

# 作成したエッジリストを保存
writecsv(list_master, "./files/list_edgelist20160127.csv")